# iOS 어플리케이션상 구동을 위한 커스텀 인식 모델 훈련

지금부터 살펴볼 코드는 간단한 음성 인식을 위한 20kb 정도의 iOS 어플리케이션용 TensorFlow Lite 모델을 훈련하는 방법을 살펴볼 것입니다. 이 모델은 micro_speech 예제 애플리케이션에서 사용된 것과 같습니다.

이 모델은 Google Colaboratory 환경에서도 사용할 수 있습니다.


[Run in Google Colab](https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/micro_speech/train_speech_model_ios_ko.ipynb)	[View source on GitHub](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/micro_speech/train_speech_model_ios_ko.ipynb)

이 노트북의 코드는 Python 스크립트를 실행하여 훈련을 진행한 뒤, 모델을 고정시킨다. 그리고 Tensorflow Lite 모델 변환기를 통해 마이크로컨트롤러용 모델을 생성한다. 

**GPU 가속을 사용하면 훈련 속도가 훨씬 빨라진다.** 훈련을 하기에 앞서, 런타임 유형을 확인해보자. **Runtime -> Change runtime type**을 클릭하면 런타임을 **GPU** 환경으로 바꿀 수 있다. 훈련에는 GPU환경을 기준으로 18,000  이터레이션의 경우 1시간반에서 2시간 정도 걸린다.


## 의존성/종속성 설치

최신버전 텐서플로우와 예제코드가 작성될 당시 사용했던 텐서플로우의 버전이 다르다
버전 잘 맞춰야 한다.

In [2]:
# !git clone https://github.com/yunho0130/tensorflow-lite.git 
!git clone https://github.com/sanghunkang/tensorflow-lite.git
!pip install tensorflow==1.15

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


Colab에서 실행중이라면, 왼쪽의 디렉토리에 tensorflow-lite가 추가된 것을 확인할 수 있다.

## 커스텀 데이터 생성 및 업로드

훈련에 앞서서 우리가 훈련에 사용하고자 하는 데이터를 생성해야 한다. speech_commands 예제에서는 1초길이의 16bit wav파일을 훈련, 테스트 및 추론에 사용하고 있다. 각 예제

ffmpeg 도구를 사용하면 변환작업을 빠르고 편리하게 수행할 수 있다.


In [ ]:
!for i in *.wav; do ffmpeg -y -i "$i" -acodec pcm_s16le -ac 1 -ar 16000 "tmp/${i%.*}.wav"; done

변환한 파일은 클래스 별로 각각 하나씩 디렉토리로 만들어서 저장한 후, 이 디렉토리들을 다시 하나의 디렉토리 안에 모아놓도록 하자. 우리는 이제 이 데이터 루트 디렉토리 경로로 지정하여 우리의 목적에 맞는 모델을 훈련할 것이다.

## 모델 훈련

이제까지는 딥러닝에서 모두가 손 대기를 꺼려하는 작업이었다. 지금부터는 딥러닝에서 재미에 해당하는 부분이다. 우리는 이제 speech_commands 예제에서 사용하는 모델의 아키텍쳐를 그대로 가져와서 그것을 우리만의 데이터셋에 적용할 것이다. 다음의 커맨드를 실행하면 모델이 훈련될 것이다.

In [3]:
!python3 tensorflow-lite/tensorflow/examples/speech_commands/train.py \
    --model_architecture=conv \
    --how_many_training_steps=10000,100  \
    --train_dir=./retrain_logs \
    --data_dir=data
    --wanted_words=bulyiya,suzy

/Library/Frameworks/Python.framework/Versions/3.7/Resources/Python.app/Contents/MacOS/Python: can't open file 'tensorflow-lite/tensorflow/examples/speech_commands/train.py': [Errno 2] No such file or directory


런타임이 모델을 훈련시키는 동안, 우리가 방금 실행시킨 커맨드가 어떤 작업을 하는지를 확인해보자.

- ```python3 tensorflow-lite/tensorflow/examples/speech_commands/train.py``` 훈련을 실행시키는 스크립트 파일
- ```--model_architecture=conv``` 훈련할 모델 아키텍쳐. speech_commands 예제에서는 ```"conv"```라고 하는 모델 아키텍쳐를 사용하였다. 사용가능한 다른 모델 아키텍쳐들의 상세한 내용은 ```tensorflow-lite/tensorflow/examples/speech_commands/models.py```에서 확인할 수 있따.
- ```--how_many_training_steps=10000,100``` 훈련을 반복할 횟수. 10000번의 가중치 업데이트 후 더 작은 learning-rate를 사용하여 100번의 추가 업데이트가 있을 것이다.
- ```--train_dir=./retrain_logs``` 훈련의 중간 결과 가중치들이 저장되는 경로. 이 결과파일들은 모델을 재훈련, 추론, 고정 및 tflite로 변환하는데 필요하다.
- ```--data_dir=data``` 앞에서 16bit .wav파일로 변환한 파일을 담고있는 폴더들의 상위폴더. 이 폴더는 bulyiya와 suzy 디렉토리를 하위 디렉토리로 가지고 있어야 한다.
- ```--wanted_words=bulyiya,suzy``` 인식하고자 하는 키워드들. ```--data_dir```에서 지정한 경로 안에 같은 이름으로 되어 훈련데이터를 가지고 있는 폴더가 각 키워드별로 있어야 한다.

만약 훈련을 중단한 모델을 중간부터 다시 훈련시키고자 한다면 ```--start_checkpoint=./retrain_logs/conv.ckpt-1100```와 같이 특정 체크포인트를 지정해서 그곳에서부터 다시 훈련을 시작할 수 있다.

이 외에도 소스코드를 직접 수정하지 않고도 여러가지 조건을 변경하여 모델을 훈련시킬 수 있다. 이에 관한 상세한 내용은 ```tensorflow-lite/tensorflow/examples/speech_commands/train.py```의 FLAGS 입력 부분에서 확인할 수 있다.

## 모델 고정

오래 기다렸다. 혹은 당신이 훌륭한 장비를 사용할 수 있는 축복받은 사람이라면 오래 기다리지 않았을 수도 있다. 이제 훈련의 결과로 생성된 체크포인트를 추론환경에서 사용하기 위하여 고정할 것이다. 훈련 중간결과를 저장하기 위한 목적인 체크포인트 파일을 고정한 .pb파일은 체크포인트파일에서 훈련에 필요한 기능들을 제거하고 최적화하여 저장할 것이다.

(주: 2.0 version부터는 saved_model이라고 하는 api를 지원한다. 기존의 pb로 만들어진 API보다 한단계 더 추상화가 된 API이다. 불행히도, pb모델과 saved_model간의 변환은 자유롭지 못하다. 그러나 다행인 것은 pb모델을 tflite모델로 변환하는 방법과 saved_model을 tflite모델로 변환하는 방법에는 크게 차이가 없다는 것이다. 따라서 본 예제의 내용흐름을 잘 따라올 수 있으면 2.x version의 텐서플로우에서도 큰 어려움 없이 tflite모델을 생성하는 파이프라인을 구성할 수 있을 것이다.)

In [12]:
%cd /Users/sanghunkang/dev/2020-contributon/
!python3 tensorflow-lite/tensorflow/examples/speech_commands/freeze.py \
    --model_architecture=conv \
    --output_file=./content/retrain_logs/conv.pb \
    --start_checkpoint=./content/retrain_logs/conv.ckpt-10100 \
    --wanted_words=bulyiya,suzy

/Users/sanghunkang/dev/2020-contributon
2020-09-13 16:40:12.060728: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-09-13 16:40:12.083694: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbce90b07b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-13 16:40:12.083722: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
{
  "desired_samples": 16000,
  "window_size_samples": 480,
  "window_stride_samples": 160,
  "spectrogram_length": 98,
  "fingerprint_width": 40,
  "fingerprint_size": 3920,
  "label_count": 4,
  "sample_rate": 16000,
  "preprocess": "mfcc",
  "average_window_width": -1
}
DECODE>>> Tensor("decoded_sample_data_1d:0", shape=(16000,), dtype=float32) DecodeWav(audio=<tf.Tensor 'decoded_sample_data:0' shape=(16000, 1) dtype=float32>, sample_rate=<tf.Tensor 'decode

## 그래프 탐색

파이썬 메모리상이 아닌 일반적인(?) 명칭으로서 우리가 훈련한 모델의 각 부분이 어떻게 되는지를 알아볼 필요가 있다. 그래프를 탐색해서 알아낸 내부구조에 특정 부분을 우리는 tflite모델을 만들면서 input과 output으로 모델 외부와 상호작용할 수 있는 채널로 개방할 것이다. 이 형식은 이후 iOS앱 개발에서 반드시 지켜줘야 하는 형식이므로 우리는 그 형식을 정확하게 iOS개발자에게 전달하기 위해서 모델의 그래프구조가 freezing 상태에서 어떻게 생겼는지 확인할 필요가 있다.

## .tflite 파일 생성

interpreter를 컴파일 한다.

In [15]:
!toco --graph_def_file=./content/retrain_logs/conv.pb \
    --output_file=./content/retrain_logs/conv.tflite \
    --input_arrays=decoded_sample_data,decoded_sample_data:1 \
    --output_arrays=labels_softmax \
    --allow_custom_ops

usage: toco [-h] --output_file OUTPUT_FILE
            (--graph_def_file GRAPH_DEF_FILE | --saved_model_dir SAVED_MODEL_DIR | --keras_model_file KERAS_MODEL_FILE)
            [--output_format {TFLITE,GRAPHVIZ_DOT}]
            [--inference_type {FLOAT,QUANTIZED_UINT8}]
            [--inference_input_type {FLOAT,QUANTIZED_UINT8}]
            [--input_arrays INPUT_ARRAYS] [--input_shapes INPUT_SHAPES]
            [--output_arrays OUTPUT_ARRAYS]
            [--saved_model_tag_set SAVED_MODEL_TAG_SET]
            [--saved_model_signature_key SAVED_MODEL_SIGNATURE_KEY]
            [--std_dev_values STD_DEV_VALUES] [--mean_values MEAN_VALUES]
            [--default_ranges_min DEFAULT_RANGES_MIN]
            [--default_ranges_max DEFAULT_RANGES_MAX]
            [--post_training_quantize] [--quantize_to_float16]
            [--drop_control_dependency] [--reorder_across_fake_quant]
            [--change_concat_input_ranges {TRUE,FALSE}] [--allow_custom_ops]
            [--target_ops TARGET_OPS]

## inference

wav로 들어올 지 bufferStream으로 들어올지를 결정하는 것은 iOS 개발자의 결정사항이지만, 일단 input레이어에 약속된대로 데이터를 꽂고 나면 output레이어에서 모델의 추론 결과를 약속한대로 뱉어내야 한다. 그것이 잘 되는지를 Python 코드 상에서 Interpreter와 예시 웨이브파일을 사용하여 확인할 것이다.

In [1]:
import numpy as np
from scipy.io import wavfile
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="content/retrain_logs/conv.tflite")
print(interpreter.get_input_details())
for tensor_detail in interpreter.get_tensor_details():
    print(tensor_detail)

# interpreter.allocate_tensors()
# print(interpreter.get_input_details()[0]["index"])
# print(interpreter.get_input_details()[1]["index"])
# print(interpreter.get_output_details()[0]["index"])
# output = interpreter.tensor(interpreter.get_output_details()[0]["index"])




# samplerate, data = wavfile.read("data/bulyiya/불이야_강상훈_1.wav")
# print(samplerate, data.shape, data.dtype, data)
# input_data = np.array(data[:16000]/32767.0, dtype=np.float32).reshape((16000, 1))
# print(input_data.shape)
# interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_data)
# interpreter.set_tensor(interpreter.get_input_details()[1]['index'], np.int32(16000))

# interpreter.invoke()
# print("inference", interpreter.get_tensor(interpreter.get_output_details()[0]['index']))

ValueError: Could not open 'content/retrain_logs/conv.tflite'.

여기까지 진행되었으면 이제 iOS에 배포할 모델의 준비는 끝났다. 모델이 준비되었다는 기쁜 소식을 iOS개발자에게 전달하고, 우리는 퇴근할 준비를 하도록 하자!